# LightGCN and NGCF
I tried applying LightGCN and NGCF to my recommendation system. Although the results were not quite great.

# Import necessary libraries

In [11]:
!pip install torch_geometric

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from abc import ABC, abstractmethod
from typing import Dict, List
import pickle
from scipy.sparse import csr_matrix
from sklearn.preprocessing import LabelEncoder
from tqdm.auto import tqdm
import networkx as nx
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch_geometric
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.utils import degree
from sklearn import preprocessing as pp
import scipy.sparse as sp

np.random.seed(42)


# Given code

In [13]:
# ACHTUNG! DO NOT TOUCH

def ndcg_metric(gt_items: np.ndarray, predicted: np.ndarray) -> float:
    at = len(predicted)
    relevance = np.array([1 if x in predicted else 0 for x in gt_items])
    # DCG uses the relevance of the recommended items
    rank_dcg = dcg(relevance)
    if rank_dcg == 0.0:
        return 0.0

    # IDCG has all relevances to 1 (or the values provided), up to the number of items in the test set that can fit in the list length
    ideal_dcg = dcg(np.sort(relevance)[::-1][:at])

    if ideal_dcg == 0.0:
        return 0.0

    ndcg_ = rank_dcg / ideal_dcg

    return ndcg_


def dcg(scores: np.ndarray) -> float:
    return np.sum(
        np.divide(np.power(2, scores) - 1, np.log2(np.arange(scores.shape[0], dtype=np.float64) + 2)), dtype=np.float64
    )


def recall_metric(gt_items: np.ndarray, predicted: np.ndarray) -> float:
    n_gt = len(gt_items)
    intersection = len(set(gt_items).intersection(set(predicted)))
    return intersection / n_gt


def evaluate_recommender(df: pd.DataFrame, model_preds_col: str, gt_col: str = "movie_id") -> Dict[str, float]:
    metric_values = []

    for _, row in df.iterrows():
        metric_values.append(
            (ndcg_metric(row[gt_col], row[model_preds_col]), recall_metric(row[gt_col], row[model_preds_col]))
        )

    return {"ndcg": np.mean([x[0] for x in metric_values]), "recall": np.mean([x[1] for x in metric_values])}

In [14]:
class BaseRecommender(ABC):
    def __init__(self):
        self.trained = False

    @abstractmethod
    def fit(self, df: pd.DataFrame) -> None:
        # реализация может быть любой, никаких ограничений
        # не забудьте про
        self.trained = True

    @abstractmethod
    def predict(self, df: pd.DataFrame, topn: int = 10) -> List[np.ndarray]:
        # реализация может быть любой, НО
        # должен возвращать список массивов из movie_id, которые есть в `item_df`, чтобы корректно работал подсчет метрик
        pass

# Data preprocessing

In [15]:
file_path = 'train_data.csv'
train_df = pd.read_csv(file_path)

file_path = 'test_data.csv' 
test_df = pd.read_csv(file_path)

train_df = train_df[(train_df['user_id'] < 1000)].copy()
le_user = pp.LabelEncoder()
le_item = pp.LabelEncoder()
train_df['user_id_idx'] = le_user.fit_transform(train_df['user_id'].values)
train_df['movie_id_idx'] = le_item.fit_transform(train_df['movie_id'].values)


train_user_ids = train_df['user_id'].unique()
train_item_ids = train_df['movie_id'].unique()

print(len(train_user_ids), len(train_item_ids))

print(len(test_df))
test_df = test_df[(test_df['user_id'].isin(train_user_ids)) & (test_df['movie_id'].isin(train_item_ids))]
print(len(test_df))

test_df['user_id_idx'] = le_user.transform(test_df['user_id'].values)
test_df['movie_id_idx'] = le_item.transform(test_df['movie_id'].values)


n_users = train_df['user_id_idx'].nunique()
n_items = train_df['movie_id_idx'].nunique()
print("Number of Unique Users : ", n_users)
print("Number of unique Items : ", n_items)

998 4483
400153
31821
Number of Unique Users :  998
Number of unique Items :  4483


# Helper functions

In [16]:
def data_loader(data, batch_size, n_usr, n_itm):
    def sample_neg(x):
        while True:
            neg_id = np.random.randint(0, n_itm)
            if neg_id not in x:
                return neg_id


    interected_items_df = data.groupby('user_id_idx')['movie_id_idx'].apply(list).reset_index()
    indices = [x for x in range(n_usr)]

    if n_usr < batch_size:
        users = [np.random.choice(indices) for _ in range(batch_size)]
    else:
        users = np.random.choice(indices, batch_size, replace=False)

    users.sort()
    users_df = pd.DataFrame(users,columns = ['users'])

    interected_items_df = pd.merge(interected_items_df, users_df, how = 'right', left_on = 'user_id_idx', right_on = 'users')
    pos_items = interected_items_df['movie_id_idx'].apply(lambda x : np.random.choice(x)).values
    neg_items = interected_items_df['movie_id_idx'].apply(lambda x: sample_neg(x)).values

    return (
        torch.LongTensor(list(users)),
        torch.LongTensor(list(pos_items)) + n_usr,
        torch.LongTensor(list(neg_items)) + n_usr
    )


In [17]:
u_t = torch.LongTensor(train_df.user_id_idx.values)
i_t = torch.LongTensor(train_df.movie_id_idx.values) + n_users
train_edge_index = torch.stack((torch.cat([u_t, i_t]), torch.cat([i_t, u_t])))
train_edge_index

tensor([[ 642,  512,  158,  ..., 2181, 3494, 1836],
        [2215, 3753, 3533,  ...,  310,  707,  153]])

In [18]:
def compute_bpr_loss(users, users_emb, pos_emb, neg_emb, user_emb0, pos_emb0, neg_emb0):
  reg_loss = (1 / 2) * (
    user_emb0.norm().pow(2) +
    pos_emb0.norm().pow(2)  +
    neg_emb0.norm().pow(2)
  ) / float(len(users))

  pos_scores = torch.mul(users_emb, pos_emb).sum(dim=1)
  neg_scores = torch.mul(users_emb, neg_emb).sum(dim=1)

  bpr_loss = torch.mean(F.softplus(neg_scores - pos_scores))

  return bpr_loss, reg_loss

In [19]:
def save_model(model, optimizer, epoch, loss, filepath='model.pkl'):
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
    }, filepath)
    
def load_model(model, optimizer, filepath='model.pkl'):
    checkpoint = torch.load(filepath)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    return epoch, loss


In [20]:
latent_dim = 64
n_layers = 10

EPOCHS = 30
BATCH_SIZE = 1024
DECAY = 0.0001
LR = 0.005
K = 10

In [21]:
def train(model, optimizer, train_df, filepath):
  loss_list_epoch = []
  bpr_loss_list_epoch = []
  reg_loss_list_epoch = []

  best_loss = float('inf')
  for epoch in tqdm(range(EPOCHS)):
      n_batch = int(len(train_df)/BATCH_SIZE)

      final_loss_list = []
      bpr_loss_list = []
      reg_loss_list = []

      model.train()
      for batch_idx in range(n_batch):

          optimizer.zero_grad()

          users, pos_items, neg_items = data_loader(train_df, BATCH_SIZE, n_users, n_items)
          users_emb, pos_emb, neg_emb, userEmb0,  posEmb0, negEmb0 = model.encode_minibatch(users, pos_items, neg_items, train_edge_index)

          bpr_loss, reg_loss = compute_bpr_loss(
            users, users_emb, pos_emb, neg_emb, userEmb0,  posEmb0, negEmb0
          )
          reg_loss = DECAY * reg_loss
          final_loss = bpr_loss + reg_loss

          final_loss.backward()
          optimizer.step()

          final_loss_list.append(final_loss.item())
          bpr_loss_list.append(bpr_loss.item())
          reg_loss_list.append(reg_loss.item())

      loss_list_epoch.append(round(np.mean(final_loss_list),4))
      bpr_loss_list_epoch.append(round(np.mean(bpr_loss_list),4))
      reg_loss_list_epoch.append(round(np.mean(reg_loss_list),4))
    
      if bpr_loss.item() < best_loss:
            best_loss = bpr_loss.item()
            save_model(model, optimizer, epoch, best_loss,  filepath)

  return (
    loss_list_epoch,
    bpr_loss_list_epoch,
    reg_loss_list_epoch,
  )


def eval(model, n_users, n_items, train_df, test_df, K):
    model.eval()
    with torch.no_grad():
      _, out = model(train_edge_index)
      final_user_Embed, final_item_Embed = torch.split(out, (n_users, n_items))
        
    test_user_ids = torch.LongTensor(test_df['user_id_idx'].unique())
    relevance_score = torch.matmul(final_user_Embed, torch.transpose(final_item_Embed,0, 1))
    i = torch.stack((
    torch.LongTensor(train_df['user_id_idx'].values),
    torch.LongTensor(train_df['movie_id_idx'].values)
    ))
    v = torch.ones((len(train_df)), dtype=torch.float64)
    interactions_t = torch.sparse.FloatTensor(i, v, (n_users, n_items))\
      .to_dense()

    relevance_score = torch.mul(relevance_score, (1 - interactions_t))

    topk_relevance_indices = torch.topk(relevance_score, K).indices
    topk_relevance_indices_df = pd.DataFrame(topk_relevance_indices.cpu().numpy(),columns =['top_indx_'+str(x+1) for x in range(K)])
    topk_relevance_indices_df['user_ID'] = topk_relevance_indices_df.index
    topk_relevance_indices_df['top_rlvnt_itm'] = topk_relevance_indices_df[['top_indx_'+str(x+1) for x in range(K)]].values.tolist()
    topk_relevance_indices_df = topk_relevance_indices_df[['user_ID','top_rlvnt_itm']]
    
    unique_user_ids = test_df['user_id_idx'].unique()
    
    all_recs = []
    
    for user_id in tqdm(unique_user_ids, desc="Predicting", unit="user"):
        recs_for_user = topk_relevance_indices_df.loc[topk_relevance_indices_df['user_ID'] == user_id, 'top_rlvnt_itm'].values[0]
        all_recs.append(recs_for_user)
    
    return all_recs


# LightGCN

### LightGCN Convolutional Layer

The LightGCN architecture is governed by the following rules:

$$e_{u}^{(k+1)} = \sum\limits_{i \in N_u} \frac{1}{\sqrt{|N_u|}\sqrt{|N_i|}}e^{(k)}_i$$

$$e_{i}^{(k+1)} = \sum\limits_{u \in N_i} \frac{1}{\sqrt{|N_i|}\sqrt{|N_u|}}e^{(k)}_u$$
In essence, the embedding for each node after a single LightGCN layer is the sum of the synthetic normalized embeddings of it's neighbors before the layer.

In [22]:
class LightGCNRecommender(BaseRecommender, nn.Module):
    class LightGCNConv(MessagePassing):
        def __init__(self, **kwargs):
            super().__init__(aggr='add')

        def forward(self, x, edge_index):
            from_, to_ = edge_index
            deg = degree(to_, x.size(0), dtype=x.dtype)
            deg_inv_sqrt = deg.pow(-0.5)
            deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
            norm = deg_inv_sqrt[from_] * deg_inv_sqrt[to_]

            return self.propagate(edge_index, x=x, norm=norm)

        def message(self, x_j, norm):
            return norm.view(-1, 1) * x_j

    def __init__(self, latent_dim, num_layers, num_users, num_items):
        super().__init__() 
        nn.Module.__init__(self)
        self.embedding = nn.Embedding(num_users + num_items, latent_dim)
        self.convs = nn.ModuleList(self.LightGCNConv() for _ in range(num_layers))
        self.init_parameters()

    def init_parameters(self):
        nn.init.normal_(self.embedding.weight, std=0.1)

    def forward(self, edge_index):
        emb0 = self.embedding.weight
        embs = [emb0]

        emb = emb0
        for conv in self.convs:
            emb = conv(x=emb, edge_index=edge_index)
            embs.append(emb)

        out = torch.mean(torch.stack(embs, dim=0), dim=0)
        return emb0, out

    def encode_minibatch(self, users, pos_items, neg_items, edge_index):
        emb0, out = self(edge_index)
        return (
            out[users],
            out[pos_items],
            out[neg_items],
            emb0[users],
            emb0[pos_items],
            emb0[neg_items]
        )

    def fit(self, optimizer, train_df: pd.DataFrame, filepath) -> None:
        train(self, optimizer, train_df, filepath)
        self.trained = True 

    def predict(self, n_users, n_items, train_df: pd.DataFrame, test_df: pd.DataFrame,topn: int = 10) -> List[np.ndarray]:
        assert self.trained
        return eval(self, n_users, n_items, train_df, test_df, topn)


In [23]:
light_gcn_recommender = LightGCNRecommender(latent_dim=latent_dim, num_layers=n_layers, num_users=n_users, num_items=n_items)
optimizer = torch.optim.Adam(light_gcn_recommender.parameters(), lr=LR)
light_gcn_recommender.fit(optimizer, train_df, 'model_light_gcn.pkl')
test_df_agg = test_df.groupby("user_id").agg({"movie_id": list}).reset_index()
test_df_agg["light_gcn_recs"] = light_gcn_recommender.predict(n_users, n_items, train_df, test_df, K)
print(evaluate_recommender(test_df_agg, model_preds_col="light_gcn_recs"))

  0%|          | 0/30 [00:00<?, ?it/s]

/var/folders/2_/xgmb5cbs7sl790jf18j7rq640000gn/T/ipykernel_2136/3067261680.py:65: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:643.)
  interactions_t = torch.sparse.FloatTensor(i, v, (n_users, n_items))\


Predicting:   0%|          | 0/978 [00:00<?, ?user/s]

{'ndcg': 0.023176415174354374, 'recall': 0.0023607010683388426}


# NGCF

### NGCF Layer

NGCF is an older architecture than LightGCN. LightGCN functions the same as NGCF, but removes the learnable linear layers, non-linear activation, and dropout.

One layer of NGCF updates user and item embeddings as follows:

$$e_{u}^{(k+1)} = \sigma\left(W_1 e_u^{(k)} + \sum\limits_{i \in N_u} \frac{1}{\sqrt{|N_u|}\sqrt{|N_i|}}(W_1e^{(k)}_i + W_2(e^{(k)}_i \odot e^{(k)}_u))\right)$$

$$e_{i}^{(k+1)} = \sigma\left(W_1 e_i^{(k)} + \sum\limits_{u \in N_i} \frac{1}{\sqrt{|N_i|}\sqrt{|N_u|}}(W_1e^{(k)}_u + W_2(e^{(k)}_u \odot e^{(k)}_i))\right)$$

Typically, NGCF is implemented with dropout before the activation and with an activation function $\sigma$ of LeakyReLU.

In [24]:
class NGCFRecommender(BaseRecommender, nn.Module):
    class NGCFConv(MessagePassing):
        def __init__(self, latent_dim, dropout, bias=True, **kwargs):
            super().__init__(aggr='add', **kwargs)
            self.dropout = dropout

            self.lin_1 = nn.Linear(latent_dim, latent_dim, bias=bias)
            self.lin_2 = nn.Linear(latent_dim, latent_dim, bias=bias)

            self.init_parameters()

        def init_parameters(self):
            nn.init.xavier_uniform_(self.lin_1.weight)
            nn.init.xavier_uniform_(self.lin_2.weight)

        def forward(self, x, edge_index):
            from_, to_ = edge_index
            deg = degree(to_, x.size(0), dtype=x.dtype)
            deg_inv_sqrt = deg.pow(-0.5)
            deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
            norm = deg_inv_sqrt[from_] * deg_inv_sqrt[to_]
            
            out = self.propagate(edge_index, x=(x, x), norm=norm)
            out += self.lin_1(x)
            out = F.dropout(out, self.dropout, self.training)
            return F.leaky_relu(out)

        def message(self, x_j, x_i, norm):
            return norm.view(-1, 1) * (self.lin_1(x_j) + self.lin_2(x_j * x_i))

    def __init__(self, latent_dim, num_layers, num_users, num_items, dropout=0.1):
        super().__init__() 
        nn.Module.__init__(self)
        self.embedding = nn.Embedding(num_users + num_items, latent_dim)
        self.convs = nn.ModuleList(self.NGCFConv(latent_dim, dropout=dropout) for _ in range(num_layers))
        self.init_parameters()

    def init_parameters(self):
        nn.init.xavier_uniform_(self.embedding.weight, gain=1)

    def forward(self, edge_index):
        emb0 = self.embedding.weight
        embs = [emb0]
        emb = emb0
    
        for conv in self.convs:
            emb = conv(x=emb, edge_index=edge_index)
            embs.append(emb)

        out = torch.cat(embs, dim=-1) 
        return emb0, out

    def encode_minibatch(self, users, pos_items, neg_items, edge_index):

        emb0, out = self(edge_index)
        return (
            out[users],
            out[pos_items],
            out[neg_items],
            emb0[users],
            emb0[pos_items],
            emb0[neg_items]
        )

    def fit(self, optimizer, train_df: pd.DataFrame, filepath) -> None:
        train(self, optimizer, train_df, filepath)
        self.trained = True 

    def predict(self, n_users, n_items, train_df: pd.DataFrame, test_df: pd.DataFrame, topn: int = 10) -> List[np.ndarray]:
        assert self.trained
        return eval(self, n_users, n_items, train_df, test_df, topn)

In [25]:
ngcf_recommender = NGCFRecommender(latent_dim=latent_dim, num_layers=n_layers, num_users=n_users, num_items=n_items)
optimizer = torch.optim.Adam(ngcf_recommender.parameters(), lr=LR)
ngcf_recommender.fit(optimizer, train_df,'model_ngcf.pkl')
test_df_agg = test_df.groupby("user_id").agg({"movie_id": list}).reset_index()
test_df_agg["ngcf_recs"] = ngcf_recommender.predict(n_users, n_items, train_df, test_df, K)
print(evaluate_recommender(test_df_agg, model_preds_col="ngcf_recs"))

  0%|          | 0/30 [00:00<?, ?it/s]

Predicting:   0%|          | 0/978 [00:00<?, ?user/s]

{'ndcg': 0.004980542170029264, 'recall': 0.00044213522094688375}
